In [2]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.credentials import Credentials
from google.oauth2 import service_account
from google.auth.transport.requests import Request
import os.path
import json
import pickle
import requests

def oauth():
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 
              'https://www.googleapis.com/auth/script.projects', 'https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/drive.file'
              ]
    creds_filename = 'test-daebong-service-account.json'  # 서비스 계정 파일 경로를 지정합니다.

    # 서비스 계정 파일을 사용하여 인증 정보를 로드합니다.
    creds = service_account.Credentials.from_service_account_file(creds_filename, scopes=SCOPES)
    return creds

def oauthByWeb():
    # 필요한 스코프를 지정합니다.
    SCOPES = [
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/script.projects',
        'https://www.googleapis.com/auth/drive',
        'https://www.googleapis.com/auth/drive.file'
    ]
    creds = None
    # 'token.json' 파일이 존재하면, 저장된 인증 정보를 불러옵니다.
    if os.path.exists('token.json'):
        with open('token.json', 'rb') as token:
            creds = pickle.load(token)

    # 저장된 인증 정보가 없거나, 유효하지 않은 경우 새로운 인증을 진행합니다.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            creds_filename = 'oauth-new-daebong.json'
            flow = InstalledAppFlow.from_client_secrets_file(creds_filename, SCOPES)
            creds = flow.run_local_server(port=8080)
        # 새로운 인증 정보를 'token.json'에 저장합니다.
        with open('token.json', 'wb') as token:
            pickle.dump(creds, token)

    return creds



In [52]:
def get_sheet_id_by_name(service, spreadsheet_id, sheet_name):
    spreadsheet_paste = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
    sheets_paste = spreadsheet_paste.get('sheets', '')
    sheet_order_id = None
    for sheet in sheets_paste:
        if sheet.get('properties', {}).get('title', '') == sheet_name:
            sheet_order_id = sheet.get('properties', {}).get('sheetId', '')
    
    return sheet_order_id

In [6]:
def get_sheet_titles(service, spreadsheet_id):
    # Google Drive API를 사용하여 파일의 메타데이터를 가져옵니다.
    file = service.files().get(fileId=spreadsheet_id, fields='name').execute()
    # 파일 이름(제목)을 반환합니다.
    return file.get('name')

In [4]:
def get_values_from_range(service, spreadsheet_id, range_name):
    # Google Sheets API를 사용하여 특정 범위의 값 가져오기
    result = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=range_name).execute()
    # 값이 있는지 확인 후 반환
    values = result.get('values', [])

    return values

In [54]:
def origin_sheet_copy_and_paste_in_specific_range(service, spreadsheet_id_copy, spreadsheet_id_paste, sheet_name, sheet_order_id):
    # A 스프레드시트의 데이터 및 서식 정보 가져오기
    response = service.spreadsheets().get(
        spreadsheetId=spreadsheet_id_copy,
        ranges=[f'{sheet_name}!A1:M8'],
        fields='sheets(data(rowData))'
    ).execute()

    # 읽은 데이터 및 서식 정보
    rows_data = response['sheets'][0]['data'][0]['rowData']

    # B 스프레드시트에 데이터 및 서식 적용
    update_requests = []
    for row_index, row in enumerate(rows_data):
        for col_index, cell in enumerate(row.get('values', [])):
            value = cell.get('userEnteredValue', {})
            format = cell.get('userEnteredFormat', {})

            # 데이터 적용 요청 추가
            update_requests.append({
                "updateCells": {
                    "rows": [
                        {
                            "values": [
                                {
                                    "userEnteredValue": value,
                                    "userEnteredFormat": format
                                }
                            ]
                        }
                    ],
                    "fields": "userEnteredValue,userEnteredFormat",
                    "range": {
                        "sheetId": sheet_order_id,
                        "startRowIndex": row_index,
                        "endRowIndex": row_index + 1,
                        "startColumnIndex": col_index,
                        "endColumnIndex": col_index + 1
                    }
                }
            })

    # 배치 업데이트로 모든 변경 적용
    response = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id_paste,
        body={"requests": update_requests}
    ).execute()

    print('Data and format copied successfully:', response)

In [55]:
def column_row_width_height_copy_and_paste_in_specific_range(service, spreadsheet_id_copy, spreadsheet_id_paste, sheet_name, sheet_order_id):


    # 열 크기 정보 가져오기 (C, D, E, L)
    column_ranges = ['C:C', 'D:D', 'E:E', 'L:L']
    columns_info = service.spreadsheets().get(
        spreadsheetId=spreadsheet_id_copy,
        ranges=[f'{sheet_name}!A1:M8'],
        fields='sheets(data(columnMetadata(pixelSize)))'
    ).execute()

    # 행 크기 정보 가져오기 (1~8)
    row_range = '1:8'
    rows_info = service.spreadsheets().get(
        spreadsheetId=spreadsheet_id_copy,
        ranges=[f'{sheet_name}!A1:M8'],
        fields='sheets(data(rowMetadata(pixelSize)))'
    ).execute()


    # 열 크기 정보 업데이트 요청
    column_update_requests = []
    for i, col in enumerate(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K',  'L', 'M']):
        pixel_size = columns_info['sheets'][0]['data'][0]['columnMetadata'][i]['pixelSize']
        column_update_requests.append({
            "updateDimensionProperties": {
                "range": {
                    "sheetId": sheet_order_id,
                    "dimension": "COLUMNS",
                    "startIndex": ord(col) - ord('A'),  # 열 문자를 인덱스로 변환
                    "endIndex": ord(col) - ord('A') + 1
                },
                "properties": {
                    "pixelSize": pixel_size
                },
                "fields": "pixelSize"
            }
        })

    # 행 크기 정보 업데이트 요청
    row_update_requests = []
    for i in range(8):  # 1~8행
        pixel_size = rows_info['sheets'][0]['data'][0]['rowMetadata'][i]['pixelSize']
        row_update_requests.append({
            "updateDimensionProperties": {
                "range": {
                    "sheetId": sheet_order_id,
                    "dimension": "ROWS",
                    "startIndex": i,
                    "endIndex": i + 1
                },
                "properties": {
                    "pixelSize": pixel_size
                },
                "fields": "pixelSize"
            }
        })

    # 모든 업데이트 요청을 배치로 실행
    update_requests = column_update_requests + row_update_requests
    response = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id_paste,
        body={"requests": update_requests}
    ).execute()
    print('Dimensions updated successfully:', response)


In [82]:
def insert_title_text_in_cell(service, spreadsheet_id, sheet_title, new_daebong_spreadsheet_title, sheet_order_id, insert_name):
    print("시트 수정")

    cell_range_c1 = 'C1'  # 수식을 적용할 셀 범위
    range_name_c1 = f"{sheet_title}!{cell_range_c1}"  # 지정된 시트의 특정 셀 범위    


    cell_range_c2 = 'C2'  # 수식을 적용할 셀 범위
    range_name_c2 = f"{sheet_title}!{cell_range_c2}"  # 지정된 시트의 특정 셀 범위    


    cell_range_e3 = 'E3'  # 수식을 적용할 셀 범위
    range_name_e3 = f"{sheet_title}!{cell_range_e3}"  # 지정된 시트의 특정 셀 범위    

    cell_range_l8 = 'L8'  # 수식을 적용할 셀 범위
    range_name_l8 = f"{sheet_title}!{cell_range_l8}"  # 지정된 시트의 특정 셀 범위    

    print(range_name_c1, range_name_c2, range_name_e3, range_name_l8)

        # 셀에 적용할 텍스트와 스타일을 설정합니다.
    text_c1 = f"대표님의 발주서 번호는 {new_daebong_spreadsheet_title}번 입니다."
    text_c2 = f"입금 시 입금자명으로 \"{insert_name}\"을 입력해주세요."
    text_e3 = f"3. 입금자명을 \"{insert_name}\"으로 입력합니다."
    text_l8 = f"{insert_name}"

    color_hex = "#2F55CB"

    
    blue = {"red": int(color_hex[1:3], 16) / 255.0, "green": int(color_hex[3:5], 16) / 255.0, "blue": int(color_hex[5:7], 16) / 255.0}  # 검은색
    black =  {"red": 0.0, "green": 0.0, "blue": 0.0}  # 검은색
    font_size_14 = 14  # 적용할 글자 크기
    font_size_12 = 12  # 적용할 글자 크기
    font_size_25 = 20  # 적용할 글자 크기
    bold = True  # 볼드 여부

    # 셀에 적용할 스타일을 설정합니다.
    text_format_c1 = {
        "foregroundColor": blue,
        "fontSize": font_size_14,
        "bold": bold
    }

    text_format_c2 = {
        "foregroundColor": blue,
        "fontSize": font_size_14,
        "bold": bold
    }

    text_format_e3 = {
        "foregroundColor": black,
        "fontSize": font_size_12,
        "bold": bold
    }

    text_format_l8 = {
        "foregroundColor": black,
        "fontSize": font_size_25,
        "bold": bold
    }


    request_1 = {
        "updateCells": {
            "rows": [
                {
                    "values": [
                        {"userEnteredValue": {"stringValue": text_c1}, "userEnteredFormat": {"textFormat": text_format_c1}}
                    ]
                }
            ],
            "fields": "userEnteredValue,userEnteredFormat.textFormat",
            "range": {
                'sheetId': sheet_order_id,
                'startRowIndex': 0, 
                'startColumnIndex': 2,
                'endRowIndex':1,
                'endColumnIndex':3
            }
        }
    }

    request_2 = {
        "updateCells": {
            "rows": [
                {
                    "values": [
                        {"userEnteredValue": {"stringValue": text_c2}, "userEnteredFormat": {"textFormat": text_format_c2}}
                    ]
                }
            ],
            "fields": "userEnteredValue,userEnteredFormat.textFormat",
            "range": {
                'sheetId': sheet_order_id,
                'startRowIndex': 1, 
                'startColumnIndex': 2, 
                'endRowIndex':2,
                'endColumnIndex':3
            }
        }
    }

    request_3 = {
        "updateCells": {
            "rows": [
                {
                    "values": [
                        {"userEnteredValue": {"stringValue": text_e3}, "userEnteredFormat": {"textFormat": text_format_e3}}
                    ]
                }
            ],
            "fields": "userEnteredValue,userEnteredFormat.textFormat",
            "range": {
                'sheetId': sheet_order_id,
                'startRowIndex': 2, 
                'startColumnIndex': 4, 
                'endRowIndex':3,
                'endColumnIndex':5
            }
        }
    }


    request_4 = {
        "updateCells": {
            "rows": [
                {
                    "values": [
                        {"userEnteredValue": {"stringValue": text_l8}, "userEnteredFormat": {"textFormat": text_format_l8}}
                    ]
                }
            ],
            "fields": "userEnteredValue,userEnteredFormat.textFormat",
            "range": {
                'sheetId': sheet_order_id,
                'startRowIndex': 7, 
                'startColumnIndex': 11, 
                'endRowIndex':8,
                'endColumnIndex':12
            }
        }
    }    

    response_1 = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body={'requests': [request_1]}
    ).execute()

    response_2 = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body={'requests': [request_2]}
    ).execute()

    response_3 = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body={'requests': [request_3]}
    ).execute()

    response_4 = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body={'requests': [request_4]}
    ).execute()

In [101]:
def optimize_sheet_operations(service, spreadsheet_id_copy, spreadsheet_id_paste, sheet_name, sheet_order_id, new_daebong_spreadsheet_title, insert_name):
    # Fetch data, row and column metadata in a single call
    full_response = service.spreadsheets().get(
        spreadsheetId=spreadsheet_id_copy,
        ranges=[f'{sheet_name}!A1:M8'],
        fields='sheets(data(rowData, columnMetadata(pixelSize), rowMetadata(pixelSize)))'
    ).execute()

    sheets_data = full_response['sheets'][0]['data'][0]
    rows_data = sheets_data.get('rowData', [])
    columns_metadata = sheets_data.get('columnMetadata', [])
    rows_metadata = sheets_data.get('rowMetadata', [])

    # Initialize requests for batch update
    update_requests = []

    # Data and formats copying
    for row_index, row in enumerate(rows_data):
        for col_index, cell in enumerate(row.get('values', [])):
            value = cell.get('userEnteredValue', {})
            format = cell.get('userEnteredFormat', {})
            update_requests.append({
                "updateCells": {
                    "rows": [
                        {
                            "values": [
                                {
                                    "userEnteredValue": value,
                                    "userEnteredFormat": format
                                }
                            ]
                        }
                    ],
                    "fields": "userEnteredValue,userEnteredFormat",
                    "range": {
                        "sheetId": sheet_order_id,
                        "startRowIndex": row_index,
                        "endRowIndex": row_index + 1,
                        "startColumnIndex": col_index,
                        "endColumnIndex": col_index + 1
                    }
                }
            })

    # Column and row dimensions copying
    for index, column in enumerate(columns_metadata):
        update_requests.append({
            "updateDimensionProperties": {
                "range": {
                    "sheetId": sheet_order_id,
                    "dimension": "COLUMNS",
                    "startIndex": index,
                    "endIndex": index + 1
                },
                "properties": {
                    "pixelSize": column['pixelSize']
                },
                "fields": "pixelSize"
            }
        })

    for index, row in enumerate(rows_metadata):
        update_requests.append({
            "updateDimensionProperties": {
                "range": {
                    "sheetId": sheet_order_id,
                    "dimension": "ROWS",
                    "startIndex": index,
                    "endIndex": index + 1
                },
                "properties": {
                    "pixelSize": row['pixelSize']
                },
                "fields": "pixelSize"
            }
        })

    # Text insertion and formatting
    texts = [
        ("C1", f"대표님의 발주서 번호는 {new_daebong_spreadsheet_title}번 입니다.", True, 14),
        ("C2", f"입금 시 입금자명으로 \"{insert_name}\"을 입력해주세요.", True, 14),
        ("E3", f"3. 입금자명을 \"{insert_name}\"으로 입력합니다.", False, 12),
        ("L8", insert_name, False, 25)
    ]
    for cell, text, is_blue, size in texts:
        color_hex = "#2F55CB" if is_blue else "#000000"
        color = {"red": int(color_hex[1:3], 16) / 255.0, "green": int(color_hex[3:5], 16) / 255.0, "blue": int(color_hex[5:7], 16) / 255.0}
        update_requests.append({
            "updateCells": {
                "rows": [
                    {
                        "values": [
                            {
                                "userEnteredValue": {"stringValue": text},
                                "userEnteredFormat": {
                                    "textFormat": {
                                        "foregroundColor": color,
                                        "fontSize": size,
                                        "bold": True
                                    }
                                }
                            }
                        ]
                    }
                ],
                "fields": "userEnteredValue,userEnteredFormat.textFormat",
                "range": {
                    'sheetId': sheet_order_id,
                    'startRowIndex': ord(cell[1]) - ord('1'),  # Convert row letter to zero-based index
                    'startColumnIndex': ord(cell[0]) - ord('A'),  # Convert column letter to zero-based index
                    'endRowIndex': ord(cell[1]) - ord('1') + 1,
                    'endColumnIndex': ord(cell[0]) - ord('A') + 1
                }
            }
        })

    # Execute all update requests in a single batch
    response = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id_paste,
        body={"requests": update_requests}
    ).execute()

    print('All operations completed successfully:', response)


In [64]:
spreadsheet_copy_id = "1REGtwSKR3A6LQa838I6myMFxKtmeDbfGfk6xh18qnmE"
spreadsheet_paste_id = "1l_dyacnX7lhxtPot1czzBiWEnLODcVuEJ77vVddfDdw"
sheet_name = "발주서"



if __name__ == "__main__":
    creds = oauthByWeb()
    service_sheets = build('sheets', 'v4', credentials=creds)
    service_drive = build("drive", "v3", credentials=creds) 

    sheet_title = get_sheet_titles(service_drive, spreadsheet_paste_id)
    # 목적지 스프레드시트 및 해당 시트 ID 찾기    
    sheet_order_id = get_sheet_id_by_name(service_sheets, spreadsheet_paste_id, sheet_name)
    origin_sheet_copy_and_paste_in_specific_range(service_sheets, spreadsheet_copy_id, spreadsheet_paste_id, sheet_name, sheet_order_id)
    column_row_width_height_copy_and_paste_in_specific_range(service_sheets, spreadsheet_copy_id, spreadsheet_paste_id, sheet_name, sheet_order_id)
    insert_title_text_in_cell(service_sheets, spreadsheet_paste_id, sheet_name, sheet_title, sheet_order_id)

Data and format copied successfully: {'spreadsheetId': '1l_dyacnX7lhxtPot1czzBiWEnLODcVuEJ77vVddfDdw', 'replies': [{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}]}
Dimensions updated successfully: {'spreadsheetId': '1l_dyacnX7lhxtPot1czzBiWEnLODcVuEJ77vVddfDdw', 'replies': [{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}]}
시트 수정
발주서!C1 발주서!C2 발주서!E3 발주서!L8


In [103]:
import time

def change_format(service_sheets, spreadsheet_copy_id, spreadsheet_paste_id, sheet_name, sheet_title, insert_name):
    
    # 목적지 스프레드시트 및 해당 시트 ID 찾기    
    sheet_order_id = get_sheet_id_by_name(service_sheets, spreadsheet_paste_id, sheet_name)
    # optimize_sheet_operations(service_sheets, spreadsheet_copy_id, spreadsheet_paste_id, sheet_name, sheet_order_id, sheet_title, insert_name)
    time.sleep(0.4)
    origin_sheet_copy_and_paste_in_specific_range(service_sheets, spreadsheet_copy_id, spreadsheet_paste_id, sheet_name, sheet_order_id)
    time.sleep(0.3)
    column_row_width_height_copy_and_paste_in_specific_range(service_sheets, spreadsheet_copy_id, spreadsheet_paste_id, sheet_name, sheet_order_id)
    time.sleep(0.2)
    insert_title_text_in_cell(service_sheets, spreadsheet_paste_id, sheet_name, sheet_title, sheet_order_id, insert_name)

    

In [107]:
user_sheet_id = "1Ajz2g-pChLSoies5umhPMcgTp5m2MIjhljJOviPlWCU"
spreadsheet_copy_id = "1REGtwSKR3A6LQa838I6myMFxKtmeDbfGfk6xh18qnmE"
import time


if __name__ == "__main__":
    creds = oauthByWeb()
    service_sheets = build('sheets', 'v4', credentials=creds)
    service_drive = build("drive", "v3", credentials=creds) 
    range_name = "셀러컨택포인트!A295:T418"
    # range_name = "셀러컨택포인트!A265:T267"
    user_data_list = get_values_from_range(service_sheets, user_sheet_id, range_name)
    print(user_data_list)

     
    for user_data in user_data_list:
        time.sleep(1)

        if user_data[10] == "":
            continue

        if user_data[1] == "24":
            continue

        print("user_data", user_data[1])
        user_id = user_data[10]
        user_insert_name = user_data[13]
        # sheet_title = get_sheet_titles(service_drive, user_id)
        sheet_title = user_data[1]

        insert_name = sheet_title

        if user_insert_name:
            insert_name = user_insert_name

        change_format(service_sheets, spreadsheet_copy_id, user_id, '발주서', sheet_title, insert_name)
        






[['', '315', '모든컴퍼니', '', '', '', '', '', '', '1035755780', '1z5OQ688lFn2nS3LIRM73ZkRBxnPirLoDfueM8xna3mg', 'https://docs.google.com/spreadsheets/d/1z5OQ688lFn2nS3LIRM73ZkRBxnPirLoDfueM8xna3mg', 'mrnoh77@gmail.com', '', '', '', '', '', '', '1'], ['', '316', '이혜련', '', '', '', '', '', '', '1048865804', '1Kzfkcfh1Fgn8TGN6UhlTT9m3IiIZ6Bo3lr_5O2jrzXo', 'https://docs.google.com/spreadsheets/d/1Kzfkcfh1Fgn8TGN6UhlTT9m3IiIZ6Bo3lr_5O2jrzXo', 'lalunar704@gmail.com', '', '', '', '', '', '', '1'], ['', '317', '여수39수산', '', '', '', '', '', '', '1048670039', '1haCXQFkgHcNmqm0z6Up9vA6H4-oYCsSZnoj42M8spUo', 'https://docs.google.com/spreadsheets/d/1haCXQFkgHcNmqm0z6Up9vA6H4-oYCsSZnoj42M8spUo', '4886km@gmail.com', '', '', '', '', '', '', '1'], ['', '318', '고대현', '', '', '', '', '', '', '1057610597', '1_4NA4bzM3zQ3JBVWG8QY4Je0BN_dWMxS-JTqCKMIb84', 'https://docs.google.com/spreadsheets/d/1_4NA4bzM3zQ3JBVWG8QY4Je0BN_dWMxS-JTqCKMIb84', 'kdh200272@gmail.com', '', '', '', '', '', '', '1'], ['', '319', '카페인컬러

In [3]:
def apply_formula_to_muti_cells(service, spreadsheet_id, range_name, phone_number):
    formulas = []
    for i in range(11, 1011):  # 11행부터 1010행까지
        formula = f'=IF(LEN(A{i})=0,"", \"{phone_number}\")'
        formulas.append([formula])

    value_range_body = {
        'values': formulas
    }

    request = service.spreadsheets().values().update(spreadsheetId=spreadsheet_id, range=range_name, valueInputOption='USER_ENTERED', body=value_range_body)
    request.execute()


In [1]:
def apply_formula_to_muti_cells_chords(service, spreadsheet_id, range_name, sheet_title):
    formulas = []
    for i in range(11, 1011):  # 11행부터 1010행까지
        formula = f'=IF(LEN(A{i})=0,"",{sheet_title})'
        formulas.append([formula])

    value_range_body = {
        'values': formulas
    }

    request = service.spreadsheets().values().update(spreadsheetId=spreadsheet_id, range=range_name, valueInputOption='USER_ENTERED', body=value_range_body)
    request.execute()


In [37]:
def apply_formula_to_muti_cells_number(service, spreadsheet_id, range_name):
    formulas = []
    for i in range(11, 1011):  # 11행부터 1010행까지
        formula = f'=IF(LEN(A{i})=0,"", 1)'
        formulas.append([formula])

    value_range_body = {
        'values': formulas
    }

    request = service.spreadsheets().values().update(spreadsheetId=spreadsheet_id, range=range_name, valueInputOption='USER_ENTERED', body=value_range_body)
    request.execute()


In [36]:
# 특정 셀에 수식을 적용하는 함수
def apply_formula_to_cell_sheet(service, spreadsheet_id, cell_range):
    formula = f'=sum(K11:K950)'  # A2 셀에 적용할 수식
    range_name = cell_range  # 지정된 시트의 특정 셀 범위
    value_range_body = {
        'values': [[formula]]  # 적용할 수식
    }
    
    request = service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        range="발주서!K8",
        valueInputOption='USER_ENTERED',
        body=value_range_body
    )
    request.execute()

In [16]:
def insert_title_text_in_cell(service, spreadsheet_id, sheet_title, sheet_order_id):
    
    cell_range_e3 = 'E2'  # 수식을 적용할 셀 범위
    range_name_e3 = f"{sheet_title}!{cell_range_e3}"  # 지정된 시트의 특정 셀 범위    

    text_e3 = f"2. 보낼 금액을 확인 후 입금합니다. "

    color_hex = "#2F55CB"

    
    blue = {"red": int(color_hex[1:3], 16) / 255.0, "green": int(color_hex[3:5], 16) / 255.0, "blue": int(color_hex[5:7], 16) / 255.0}  # 검은색
    black =  {"red": 0.0, "green": 0.0, "blue": 0.0}  # 검은색
    font_size_14 = 14  # 적용할 글자 크기
    font_size_12 = 12  # 적용할 글자 크기
    font_size_25 = 20  # 적용할 글자 크기
    bold = True  # 볼드 여부

    text_format_e3 = {
        "foregroundColor": black,
        "fontSize": font_size_12,
        "bold": bold
    }

    request_3 = {
        "updateCells": {
            "rows": [
                {
                    "values": [
                        {"userEnteredValue": {"stringValue": text_e3}, "userEnteredFormat": {"textFormat": text_format_e3}}
                    ]
                }
            ],
            "fields": "userEnteredValue,userEnteredFormat.textFormat",
            "range": {
                'sheetId': sheet_order_id,
                'startRowIndex': 1, 
                'startColumnIndex': 4, 
                'endRowIndex':2,
                'endColumnIndex':5
            }
        }
    }

    response_3 = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body={'requests': [request_3]}
    ).execute()


In [47]:
# 색상 코드를 RGB 소수 값으로 변환
def hex_to_rgb_percent(hex):
    hex = hex.lstrip('#')
    return tuple(int(hex[i:i+2], 16)/255.0 for i in (0, 2, 4))

def background_color_change(service, spreadsheet_id, sheetId):
    # 배경색 설정
    background_color = hex_to_rgb_percent("7AD592")
    requests = [
    # A1 셀의 배경색 설정 및 텍스트 삭제
    {
        'repeatCell': {
            'range': {
                'sheetId': sheetId,
                'startRowIndex': 8,
                'endRowIndex': 9,
                'startColumnIndex': 8,
                'endColumnIndex': 9
            },
            'cell': {
                'userEnteredFormat': {
                    'backgroundColor': {
                        'red': background_color[0],
                        'green': background_color[1],
                        'blue': background_color[2]
                    }
                },
                'userEnteredValue': {}  # 텍스트를 삭제
            },
            'fields': 'userEnteredFormat.backgroundColor, userEnteredValue'
        }
    },
    # B1 셀의 배경색 설정
    {
        'repeatCell': {
            'range': {
                'sheetId': sheetId,
                'startRowIndex': 9,
                'endRowIndex': 10,
                'startColumnIndex': 8,
                'endColumnIndex': 9
            },
            'cell': {
                'userEnteredFormat': {
                    'backgroundColor': {
                        'red': background_color[0],
                        'green': background_color[1],
                        'blue': background_color[2]
                    }
                }
            },
            'fields': 'userEnteredFormat.backgroundColor'
        }
    }
]


    # 요청 실행
    body = {
        'requests': requests
    }
    response = service.spreadsheets().batchUpdate(spreadsheetId=spreadsheet_id, body=body).execute()

    print('Response:', response)

In [6]:
user_sheet_id = "1Ajz2g-pChLSoies5umhPMcgTp5m2MIjhljJOviPlWCU"
spreadsheet_copy_id = "1REGtwSKR3A6LQa838I6myMFxKtmeDbfGfk6xh18qnmE"
import time


if __name__ == "__main__":
    creds = oauthByWeb()
    service_sheets = build('sheets', 'v4', credentials=creds)
    service_drive = build("drive", "v3", credentials=creds) 
    # range_name = "셀러컨택포인트!A305:T330"
    range_name = "셀러컨택포인트!A331:T332"
    user_data_list = get_values_from_range(service_sheets, user_sheet_id, range_name)
    print(user_data_list)

    origin_sheet_id = "1REGtwSKR3A6LQa838I6myMFxKtmeDbfGfk6xh18qnmE"

    origin_phone_number = "01051319792"

    # apply_formula_to_muti_cells(service_sheets, origin_sheet_id, "발주서!I11:I", origin_phone_number)

     
    for user_data in user_data_list:
        time.sleep(1)

        if user_data[10] == "":
            continue

        if user_data[1] == "24":
            continue

        phone_number = "01051319792"
        if user_data[9]:
            phone_number = user_data[9]
            

        print("user_data", user_data[1])
        user_id = user_data[10]
        user_insert_name = user_data[13]
        # sheet_title = get_sheet_titles(service_drive, user_id)
        sheet_title = user_data[1]

        insert_name = sheet_title

        if user_insert_name:
            insert_name = user_insert_name

        
        apply_formula_to_muti_cells_chords(service_sheets, user_id, "발주서!J11:J", sheet_title)
        # apply_formula_to_muti_cells_number(service_sheets, user_id, "발주서!D11:D")
        # sheet_order_id = get_sheet_id_by_name(service_sheets, user_id, "발주서")
        # background_color_change(service_sheets, user_id, sheet_order_id)
        



[['', '349', '해피상회', '', '', '', '', '', '', '1041677881', '1WqF0xezvhkCro5f7aIn3yhmmD8x5fve4sFC18qa8yn4', 'https://docs.google.com/spreadsheets/d/1WqF0xezvhkCro5f7aIn3yhmmD8x5fve4sFC18qa8yn4', 'ymee788@gmail.com', '', '', '', '', '', '', '1'], ['', '350', '소담유통', '', '', '', '', '', '', '1076890334', '19ncCK2cIQG4Yu3j4AjhzHbyRHadPnU5LBeJ1Pu-q9wg', 'https://docs.google.com/spreadsheets/d/19ncCK2cIQG4Yu3j4AjhzHbyRHadPnU5LBeJ1Pu-q9wg', 'moonseungsu123@gmail.com', '', '', '', '', '', '', '1']]
user_data 349
user_data 350


In [ ]:
user_id = "1REGtwSKR3A6LQa838I6myMFxKtmeDbfGfk6xh18qnmE"


if __name__ == "__main__":
    creds = oauthByWeb()
    phone_number = "01051319792"
    service_sheets = build('sheets', 'v4', credentials=creds)
    apply_formula_to_muti_cells(service_sheets, user_id, "발주서!I11:I", phone_number)

In [50]:
def aws_apply_formula_to_muti_cells_phone_number(service, spreadsheet_id, range_name, phone_number):
    formulas = []
    for i in range(11, 1011):  # 11행부터 1010행까지
        formula = f'=IF(LEN(A{i})=0,"", \"{phone_number}\")'
        formulas.append([formula])

    value_range_body = {
        'values': formulas
    }

    request = service.spreadsheets().values().update(spreadsheetId=spreadsheet_id, range=range_name, valueInputOption='USER_ENTERED', body=value_range_body)
    request.execute()


In [51]:
def aws_insert_title_text_in_cell(service, spreadsheet_id, sheet_order_id, insert_name):
    text_c2 = f"입금 시 입금자명으로 \"{insert_name}\"을 입력해주세요."
    text_e3 = f"3. 입금자명을 \"{insert_name}\"으로 입력합니다."
    text_l8 = f"{insert_name}"

    color_hex = "#2F55CB"

    
    blue = {"red": int(color_hex[1:3], 16) / 255.0, "green": int(color_hex[3:5], 16) / 255.0, "blue": int(color_hex[5:7], 16) / 255.0}  # 검은색
    black =  {"red": 0.0, "green": 0.0, "blue": 0.0}  # 검은색
    font_size_14 = 14  # 적용할 글자 크기
    font_size_12 = 12  # 적용할 글자 크기
    font_size_25 = 20  # 적용할 글자 크기
    bold = True  # 볼드 여부

    text_format_c2 = {
        "foregroundColor": blue,
        "fontSize": font_size_14,
        "bold": bold
    }

    text_format_e3 = {
        "foregroundColor": black,
        "fontSize": font_size_12,
        "bold": bold
    }

    text_format_l8 = {
        "foregroundColor": black,
        "fontSize": font_size_25,
        "bold": bold
    }

    request_2 = {
        "updateCells": {
            "rows": [
                {
                    "values": [
                        {"userEnteredValue": {"stringValue": text_c2}, "userEnteredFormat": {"textFormat": text_format_c2}}
                    ]
                }
            ],
            "fields": "userEnteredValue,userEnteredFormat.textFormat",
            "range": {
                'sheetId': sheet_order_id,
                'startRowIndex': 1, 
                'startColumnIndex': 2, 
                'endRowIndex':2,
                'endColumnIndex':3
            }
        }
    }

    request_3 = {
        "updateCells": {
            "rows": [
                {
                    "values": [
                        {"userEnteredValue": {"stringValue": text_e3}, "userEnteredFormat": {"textFormat": text_format_e3}}
                    ]
                }
            ],
            "fields": "userEnteredValue,userEnteredFormat.textFormat",
            "range": {
                'sheetId': sheet_order_id,
                'startRowIndex': 2, 
                'startColumnIndex': 4, 
                'endRowIndex':3,
                'endColumnIndex':5
            }
        }
    }


    request_4 = {
        "updateCells": {
            "rows": [
                {
                    "values": [
                        {"userEnteredValue": {"stringValue": text_l8}, "userEnteredFormat": {"textFormat": text_format_l8}}
                    ]
                }
            ],
            "fields": "userEnteredValue,userEnteredFormat.textFormat",
            "range": {
                'sheetId': sheet_order_id,
                'startRowIndex': 7, 
                'startColumnIndex': 11, 
                'endRowIndex':8,
                'endColumnIndex':12
            }
        }
    }    

    response_2 = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body={'requests': [request_2]}
    ).execute()

    response_3 = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body={'requests': [request_3]}
    ).execute()

    response_4 = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body={'requests': [request_4]}
    ).execute()

In [53]:
def aws_get_sheet_id_by_name(service, spreadsheet_id, sheet_name):
    spreadsheet_paste = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
    sheets_paste = spreadsheet_paste.get('sheets', '')
    sheet_order_id = None
    for sheet in sheets_paste:
        if sheet.get('properties', {}).get('title', '') == sheet_name:
            sheet_order_id = sheet.get('properties', {}).get('sheetId', '')
    
    return sheet_order_id

In [54]:
def lambda_handler(event, context): 
    creds = oauth()
    service = build('sheets', 'v4', credentials=creds)
    spreadsheet_id = event["spreadsheet_id"]
    insert_name = event["insert_name"]
    phone_number = event["phone_number"]
    sheet_order_id = aws_get_sheet_id_by_name(service, spreadsheet_id, "발주서")

    try: 
        if phone_number:
            aws_apply_formula_to_muti_cells_phone_number(service, spreadsheet_id, "발주서!I11:I", phone_number)
        if insert_name:
            aws_insert_title_text_in_cell(service, spreadsheet_id, sheet_order_id, insert_name)
    except HttpError as error:
        # HttpError에서 응답 코드와 메시지를 추출합니다.
        error_code = error.resp.status
        try:
            # 오류 응답 본문을 JSON 객체로 파싱합니다.
            error_details = json.loads(error.content.decode())
            error_message = error_details.get('error', {}).get('message', 'Unknown error')
        except json.JSONDecodeError:
            # 오류 응답 본문이 JSON이 아닌 경우, 일반 텍스트로 처리합니다.
            error_message = error.content.decode()

        print(f"An error occurred: {error}")
        return {
            'statusCode': error_code,  # HTTP 상태 코드를 응답에 설정합니다.
            'body': json.dumps({"error": error_message})  # 오류 메시지를 JSON 응답에 포함시킵니다.
        }

    return {
        'statusCode': 200,
        'body': json.dumps({"result": "Success", "message": "Formula applied successfully"})
    }

In [56]:
if __name__ == "__main__":

    lambda_handler({
        "spreadsheet_id": "1RAnWFmNvlv2VUk_biNbpUyhasr6o2Gt5HXZEk216sDs",
        "insert_name": "338_백민기",
        "phone_number":"01052371951"
    }, {})
